## Distribute Workload using ZeroMQ

In this section, we will be using ZeroMQ to distribute workload of generating temporal traces
among distributed workers. Doing so, gives us the ability to get a much higher throughput of
simulations.

In [1]:
import sys
import time
import random
from threading import Thread
import struct
import pickle

import zmq
from tqdm import tqdm
import pandas as pd

port = "5556"


context = zmq.Context()
socket = context.socket(zmq.DEALER)
socket.setsockopt(zmq.IDENTITY, b'master')
socket_addr = "tcp://127.0.0.1:%s" % port
socket.bind(socket_addr)

poller = zmq.Poller()
poller.register(socket, zmq.POLLIN)

In [2]:
# Test API function before running worker
from sim_trace import generate_trace_api

data = {
    "arrival_rate": 0.9, 
    "warm_service_rate": 1/2.016, 
    "cold_service_rate": 1/2.163,
    "expiration_threshold": 600, 
    "max_time": 300,
}

generate_trace_api(data)

{'reqs_cold': 7,
 'reqs_total': 259,
 'reqs_warm': 252,
 'prob_cold': 0.02702702702702703,
 'reqs_reject': 0,
 'prob_reject': 0.0,
 'lifespan_avg': nan,
 'inst_count_avg': 6.373843743298862,
 'inst_running_count_avg': 1.8784727180748448,
 'inst_idle_count_avg': 4.495371025224019,
 'arrival_rate': 0.9,
 'warm_service_rate': 0.49603174603174605,
 'cold_service_rate': 0.46232085067036527,
 'expiration_threshold': 600,
 'max_time': 300}

In [3]:
total_sims = 10000

# generate request that will be sent out to the worker
request = pickle.dumps(data)
def sender(num):
    # send the tasks
    for _ in range(num):
        socket.send(request)

st = Thread(target=sender, args=(total_sims,))
st.start()

In [4]:
pbar = tqdm(total=int(total_sims))

received_sims = 0
results = []
# receive the results
while received_sims < total_sims:
    socks = dict(poller.poll(timeout=5000))
    if socks == {}:
        print("Timeout!")
        break

    if socket in socks and socks[socket] == zmq.POLLIN:
        # print("Message from socket: %s" % struct.unpack("d", socket.recv()))
        results.append(pickle.loads(socket.recv()))
        received_sims += 1
        pbar.update(1)

pbar.close()

100%|██████████| 10000/10000 [00:28<00:00, 346.43it/s]


In [5]:
res_df = pd.DataFrame(data=results)
print(f"The probability of cold start is: {res_df['prob_cold'].mean():.8f}")

res_df.head()

The probability of cold start is: 0.02675923


,reqs_cold,reqs_total,reqs_warm,prob_cold,reqs_reject,prob_reject,lifespan_avg,inst_count_avg,inst_running_count_avg,inst_idle_count_avg,arrival_rate,warm_service_rate,cold_service_rate,expiration_threshold,max_time
0,6,290,284,0.020690,0,0.0,NaN,5.288575,1.849553,3.439021,0.9,0.496032,0.462321,600,300
1,7,251,244,0.027888,0,0.0,NaN,6.267992,1.763723,4.504269,0.9,0.496032,0.462321,600,300
2,6,266,260,0.022556,0,0.0,NaN,5.768675,1.651955,4.116720,0.9,0.496032,0.462321,600,300
3,7,254,247,0.027559,0,0.0,NaN,5.454274,1.639920,3.814354,0.9,0.496032,0.462321,600,300
4,6,251,245,0.023904,0,0.0,NaN,5.270242,1.552482,3.717760,0.9,0.496032,0.462321,600,300


In [6]:
res_df.mean().to_dict()

{'reqs_cold': 7.2316,
 'reqs_total': 270.6715,
 'reqs_warm': 263.4399,
 'prob_cold': 0.026759232023040664,
 'reqs_reject': 0.0,
 'prob_reject': 0.0,
 'lifespan_avg': nan,
 'inst_count_avg': 6.35760382595932,
 'inst_running_count_avg': 1.805940194580486,
 'inst_idle_count_avg': 4.551663631378818,
 'arrival_rate': 0.899999999999831,
 'warm_service_rate': 0.4960317460317861,
 'cold_service_rate': 0.4623208506703655,
 'expiration_threshold': 600.0,
 'max_time': 300.0}